# Cox-Ross-Rubinstein Model

In [ ]:
import numpy as np
from scipy.stats import lognorm
#
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
class CoxRossRubinsteinModel:

    def __init__(self, S0, r, d, u, T, N):
        self.S0 = S0  # asset value at t=0
        self.r  = r   # constant risk-free rate
        self.d  = d   # relative down move
        self.u  = u   # relative up move
        self.T  = T   # terminal time
        self.N  = N   # number of time steps from 0 to T
        #
        self.times = np.linspace(0.0, self.T, self.N+1)
        self.gridPoints = [ np.array([ 1.0 ]) ]
        for k in range(1,self.N+1):
            self.gridPoints.append(np.array([ (1+d)**(k-i) * (1+u)**i for i in range(k+1) ]))

    def moves(self, depth=None):
        if depth == None:
            depth = self.N
        if depth == 1:
            return np.array([ [ self.d ], [ self.u ] ])
        subTree = self.moves(depth-1)
        D = np.append( self.d * np.ones([subTree.shape[0],1]), subTree, axis=1)
        U = np.append( self.u * np.ones([subTree.shape[0],1]), subTree, axis=1)
        return np.append(D, U, axis=0)
    
    def paths(self):
        m = self.moves()
        p = np.ones([m.shape[0], m.shape[1]+1])
        for k in range(self.N):
            p[:,k+1] = p[:,k] * (1+m[:,k])
        return p

    def blackScholesParameters(self):
        """Derive approximate Black Scholes parameters"""
        continuousRate = np.log(1+self.r) * self.N / self.T
        # there is a priori no guarantee that our tree is symmetric
        sigma_d = -np.log(1+self.d) * np.sqrt(self.N / self.T)
        sigma_u =  np.log(1+self.u) * np.sqrt(self.N / self.T)
        averageSigma = np.sqrt(sigma_d * sigma_u)
        return (continuousRate, averageSigma)

    def blackScholesPdf(self, S):
        """Calculate the BS pdf function"""
        r, sigma = self.blackScholesParameters()
        return lognorm.pdf(S, sigma, r*self.T)


    def plot(self, withDensity=False):
        fig = make_subplots(rows=1, cols=2, shared_xaxes=True, vertical_spacing=0.02)
        fig.update_layout(
            autosize=False,
            width=800,
            height=500,
        )
        #
        paths_ = self.paths()
        for p in paths_:
            fig.add_trace(go.Scatter(x=self.times, y=p, mode='lines', line=dict(width=1, color='Blue')),
                row=1, col=1)
        #
        for t, points in zip(self.times, self.gridPoints):
            fig.add_trace(go.Scatter(x=t*np.ones(points.shape), y=points, mode='markers', marker=dict(size=5, color='Red')),
                row=1, col=1)
        #
        if withDensity:
            # only works well for symmetric tree and with zero r
            fig.add_trace(go.Histogram(y=paths_[:,-1], histnorm='probability density', nbinsy=self.N-1), row=1, col=2)
            #
            r, sigma = self.blackScholesParameters()
            S = np.exp(np.linspace(-3*sigma*np.sqrt(self.T), 3*sigma*np.sqrt(self.T), 100))
            pdf = self.blackScholesPdf(S)
            fig.add_trace(go.Scatter(y=S, x=pdf), row=1, col=2)
        else:
            fig.add_trace(go.Histogram(y=paths_[:,-1], histnorm='probability'), row=1, col=2)
        fig.update_layout(showlegend=False)
        fig.show()

m = CoxRossRubinsteinModel(100, 0.00, -0.05, 0.10, 1.0, 10)

In [ ]:
CoxRossRubinsteinModel(100, 0.01, -0.05, 0.10, 1.0, 10).plot(withDensity=False)

In [ ]:
CoxRossRubinsteinModel(100, 0.00, -0.05, 0.05, 1.0, 10).plot(withDensity=True)